## We need the optimal strategy to load/save models

In [12]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.expandvars("/home/$USER/repos/fande/") )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import torch
from fande.ase import FandeCalc

predictor_loaded = torch.load("/data1/simulations/ML_models/predictor_nonforced_last.pt")
fande_calc = FandeCalc(predictor_loaded)

atoms = predictor_loaded.fdm.traj_train[0].copy()
atoms.calc = fande_calc

print( atoms.get_potential_energy(), atoms.get_forces() )


Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:04<00:00,  4.77s/it]


0.0 [[-0.05307007 -0.07954407  0.06188965]
 [ 0.02107239  0.04133606 -0.0723114 ]
 [-0.06694031  0.04341125  0.00621033]
 ...
 [ 0.07273865  0.21099091 -0.20579529]
 [-0.01686096 -0.02398682 -0.05099869]
 [ 0.02691841 -0.07848358  0.10313416]]


In [19]:
atoms.positions[0,0] += 0.001
print( atoms.get_potential_energy(), atoms.get_forces() )

Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.64s/it]


0.0 [[-0.2003479  -0.08526611  0.05366516]
 [ 0.02111816  0.04136658 -0.07221985]
 [-0.06692505  0.04341125  0.00617981]
 ...
 [ 0.07267761  0.2109375  -0.20581436]
 [-0.01685333 -0.02397919 -0.05105591]
 [ 0.02691078 -0.07852936  0.10313416]]


In [22]:
print(len(predictor_loaded.fdm.train_indices[5]) )
print(predictor_loaded.fdm.soap_hypers)

10000
{'soap_type': 'PowerSpectrum', 'interaction_cutoff': 6.0, 'max_radial': 4, 'max_angular': 4, 'gaussian_sigma_constant': 0.3, 'gaussian_sigma_type': 'Constant', 'cutoff_function_type': 'RadialScaling', 'cutoff_smooth_width': 0.1, 'cutoff_function_parameters': {'rate': 1, 'scale': 3.5, 'exponent': 4, 'interaction_cutoff': 6.0, 'cutoff_smooth_width': 0.1}, 'radial_basis': 'GTO', 'normalize': True, 'compute_gradients': True, 'expansion_by_species_method': 'structure wise'}


In [23]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
traj_355 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")

traj_295_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
traj_355_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
traj_295_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
traj_355_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")

In [24]:
from fande.utils.test import make_test_plots

In [25]:
save_dir_295 = "/data1/simulations/ML_models/test_plots/295"
save_dir_355 = "/data1/simulations/ML_models/test_plots/355"
save_dir_295_2000 = "/data1/simulations/ML_models/test_plots/295_2000"
save_dir_355_2000 = "/data1/simulations/ML_models/test_plots/295_2000"
save_dir_295_2000_forced = "/data1/simulations/ML_models/test_plots/295_2000_forced"
save_dir_355_2000_forced = "/data1/simulations/ML_models/test_plots/355_2000_forced"

In [26]:
trajectories = [traj_295, traj_355, traj_295_2000K, traj_355_2000K, traj_295_2000K_forced, traj_355_2000K_forced]
save_dirs = [save_dir_295, save_dir_355, save_dir_295_2000, save_dir_355_2000, save_dir_295_2000_forced, save_dir_355_2000_forced]

In [27]:
%%capture c

for traj, save_dir in zip(trajectories, save_dirs):
    make_test_plots(fande_calc, traj, save_dir=save_dir)

KeyboardInterrupt: 

: 